In [37]:
import pandas as pd

In [38]:
orders = pd.read_csv("C:/Users/belou/OneDrive/Рабочий стол/пет проект/archive/orders.csv")
customers = pd.read_csv("C:/Users/belou/OneDrive/Рабочий стол/пет проект/archive/customers.csv")
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [46]:
df = orders.merge( #соединение таблиц 
    customers, 
    on='customer_id', 
    how='inner'
)
df = df[df['order_status'] == 'delivered']
df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,barreiras,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP


In [47]:
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])#формат
df['order_purchase_timestamp'].dtype #проверить формат 

dtype('<M8[ns]')

In [48]:
df['order_date'] = df['order_purchase_timestamp'].dt.date #создали столбец дата закзаза 
df['start_date'] = (
    df
    .groupby('customer_unique_id')['order_date'] #группируем и берем столбец 
    .transform('min') #аналог окна, дф не меняется, просто рядом добавляется столбик 
)#дата первого заказа 
df[['customer_unique_id', 'order_date', 'start_date']].head()

,customer_unique_id,order_date,start_date
0,7c396fd4830fd04220f754e42b4e5bff,2017-10-02,2017-09-04
1,af07308b275d755c9edb36a90c618231,2018-07-24,2018-07-24
2,3a653a41f6f9fc3d2a113cf8398680e8,2018-08-08,2018-08-08
3,7c142cf63193a1473d2e66489a9ae977,2017-11-18,2017-11-18
4,72632f0f9dd73dfee390c9b22eb56dd6,2018-02-13,2018-02-13


In [49]:
df['order_month'] = df['order_date'].apply(lambda x: x.strftime('%Y-%m')) #оставляем только месяц и год, аплай-берет каждое значение и применяет функцию к нему
df['start_month'] = df['start_date'].apply(lambda x: x.strftime('%Y-%m')) #лямбда - просто название функции,может быть любое, те каждое икс привести к такому формату 
df[['order_date', 'order_month', 'start_date', 'start_month']].head() 

,order_date,order_month,start_date,start_month
0,2017-10-02,2017-10,2017-09-04,2017-09
1,2018-07-24,2018-07,2018-07-24,2018-07
2,2018-08-08,2018-08,2018-08-08,2018-08
3,2017-11-18,2017-11,2017-11-18,2017-11
4,2018-02-13,2018-02,2018-02-13,2018-02


In [50]:
df['order_year'] = df['order_month'].apply(lambda x: int(x.split('-')[0])) #отделяем годы и месяцы от даты 
df['order_mn'] = df['order_month'].apply(lambda x: int(x.split('-')[1]))
df['start_year'] = df['start_month'].apply(lambda x: int(x.split('-')[0]))
df['start_mn'] = df['start_month'].apply(lambda x: int(x.split('-')[1]))
df['months_diff'] = (
    (df['order_year']-df['start_year'])*12  + (df['order_mn']-df['start_mn']) 
)
df[['start_month', 'order_month', 'months_diff']].head()


,start_month,order_month,months_diff
0,2017-09,2017-10,1
1,2018-07,2018-07,0
2,2018-08,2018-08,0
3,2017-11,2017-11,0
4,2018-02,2018-02,0


In [51]:
cohort_size = df[df['months_diff']==0].groupby('start_month')['customer_unique_id'].nunique() 
retention_counts = df.groupby(['start_month', 'months_diff'])['customer_unique_id'].nunique()
retention = retention_counts/cohort_size
retention = retention.reset_index()
retention.rename(columns={'customer_unique_id': 'retention'}, inplace=True)
retention['retention'] = retention['retention'] * 100
retention = retention[retention['months_diff'] == 1]
retention['retention'] = retention['retention'].round(2)
retention.head(10)


,start_month,months_diff,retention
11,2016-12,1,100.00
13,2017-01,1,0.28
31,2017-02,1,0.18
49,2017-03,1,0.44
67,2017-04,1,0.62
84,2017-05,1,0.46
100,2017-06,1,0.49
115,2017-07,1,0.53
129,2017-08,1,0.69
142,2017-09,1,0.70
